In [4]:
import pandas as pd
from xml.etree import ElementTree as ET
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

content_dir = '/home/bbb1417/LLM_SR_medicine/reviews_data/PICOS/'
file_path = '/home/bbb1417/LLM_SR_medicine/reviews_data/PICOS/All_incl_articles_20230630.xml'
tree = ET.parse(file_path)
root = tree.getroot()

FileNotFoundError: [Errno 2] No such file or directory: '/home/bbb1417/LLM_SR_medicine/reviews_data/PICOS/All_incl_articles_20230630.xml'

In [ ]:

def get_all_text(element, default=None):
    """Recursively get all text in the element and its descendants."""
    if element is not None:
        return ''.join(element.itertext())
    return default

def parse_record(record):
    """Parse a single record and return a dictionary."""
    row = {}
    for child in record.iter():
        tag = child.tag
        text = get_all_text(child)
        row[tag] = text
    return row

data = [parse_record(record) for record in root.findall('./records/record')]
df = pd.DataFrame(data)
df

In [2]:

# We'll use TF-IDF to vectorize the lines and DataFrame text, then use cosine similarity to find the best matches.
df['concatenated_fields'] = df[[ 'contributors', 'titles', 'title','secondary-title', 'short-title','pages', 'volume', 'number',
       'edition', 'keywords', 'keyword', 'dates', 'year', 'pub-dates', 'date',
       'isbn', 'accession-num', 'authors',
       'author',]].fillna('').astype(str).apply(lambda x: ' '.join(x), axis=1)

pattern = r'(enlEndNote\d+[A-Za-z]+)'
print(len(df))
# Finding duplicated rows
df.drop_duplicates()# Showing duplicated rows
print(len(df))
df['uniqueid'] = df['record'].str.extract(pattern)
print(len(df['uniqueid'].unique()))
duplicates = df[df.duplicated(subset='uniqueid')]
df=df.drop_duplicates(subset='uniqueid', keep='first')

pattern = r'(enlEndNote\d+)'
duplicates['uniqueid'] = duplicates['record'].str.extract(pattern)
print(len(duplicates['uniqueid'].unique()))
df=pd.concat([df,duplicates])
print(len(df['uniqueid'].unique()))

df = df.drop_duplicates(subset='uniqueid', keep='first')
print(len(df['uniqueid'].unique()))

# Create a TF-IDF Vectorizer
vectorizer = TfidfVectorizer(min_df=1, analyzer='word', stop_words='english')


# Re-load the corrected text file since the variable was lost
with open(f'{content_dir}screening2_info/corrected_excluded_screening2.txt', 'r') as file:
    corrected_lines = file.readlines()

# Strip whitespace for a clean comparison
corrected_lines = [line.strip() for line in corrected_lines]


# We'll use TF-IDF to vectorize the lines and DataFrame text, then use cosine similarity to find the best matches.

# Create a TF-IDF Vectorizer
vectorizer = TfidfVectorizer(min_df=1, analyzer='word', stop_words='english')

# Vectorize the concatenated fields of the DataFrame
tfidf_matrix_df = vectorizer.fit_transform(df['concatenated_fields'])

# Vectorize the lines from the text file
tfidf_matrix_lines = vectorizer.transform(corrected_lines)

# Calculate the cosine similarity between each line and each row in the DataFrame
cosine_similarities = cosine_similarity(tfidf_matrix_lines, tfidf_matrix_df)

# For each line, find the index of the row with the highest cosine similarity
best_match_indices = np.argmax(cosine_similarities, axis=1)

# Get the similarity scores for the best matches
best_match_scores = cosine_similarities[np.arange(cosine_similarities.shape[0]), best_match_indices]

# Pair each line with its best matching row index and similarity score
matching_results = list(zip(best_match_indices, best_match_scores))



# Create a dataframe to hold the results for better visualization
matches_df = pd.DataFrame({
    'LineIndex': np.arange(len(corrected_lines)),
    'BestMatchRowIndex': best_match_indices,
    'SimilarityScore': best_match_scores
})

# Add the corresponding line and the matching text from the dataframe to the results dataframe
matches_df['Line'] = [corrected_lines[i] for i in matches_df['LineIndex']]
matches_df['BestMatchText'] = [df.iloc[i]['concatenated_fields'] for i in matches_df['BestMatchRowIndex']]

matches_df.loc[matches_df['LineIndex'] == 64, 'BestMatchRowIndex'] = 252
matches_df.loc[matches_df['LineIndex'] == 16, 'BestMatchRowIndex'] = 1414
matches_df[matches_df.BestMatchRowIndex.duplicated(keep=False)]

df['screening1'] = False
df['screening2'] = False

df['screening1'] = df.index.isin(matches_df['BestMatchRowIndex'])
df.screening1.value_counts()


with open(f'{content_dir}screening1_info/75_included.txt', 'r') as file:
    included75 = file.read().split('\n')

with open(f'{content_dir}screening1_info/citations_included.txt', 'r') as file:
    citations_review = file.read().split('\n')

# Create a TF-IDF Vectorizer
vectorizer = TfidfVectorizer(min_df=1, analyzer='word', stop_words='english')
# Strip whitespace for a clean comparison
corrected_lines = [line.strip() for line in citations_review]
# We'll use TF-IDF to vectorize the lines and DataFrame text, then use cosine similarity to find the best matches.

# Create a TF-IDF Vectorizer
vectorizer = TfidfVectorizer(min_df=1, analyzer='word', stop_words='english')

# Vectorize the concatenated fields of the DataFrame
tfidf_matrix_df = vectorizer.fit_transform(df['concatenated_fields'])

# Vectorize the lines from the text file
tfidf_matrix_lines = vectorizer.transform(citations_review)

# Calculate the cosine similarity between each line and each row in the DataFrame
cosine_similarities = cosine_similarity(tfidf_matrix_lines, tfidf_matrix_df)

# For each line, find the index of the row with the highest cosine similarity
best_match_indices = np.argmax(cosine_similarities, axis=1)

# Get the similarity scores for the best matches
best_match_scores = cosine_similarities[np.arange(cosine_similarities.shape[0]), best_match_indices]

# Pair each line with its best matching row index and similarity score
matching_results = list(zip(best_match_indices, best_match_scores))


# Create a dataframe to hold the results for better visualization
matches_df = pd.DataFrame({
    'LineIndex': np.arange(len(citations_review)),
    'BestMatchRowIndex': best_match_indices,
    'SimilarityScore': best_match_scores
})

# Add the corresponding line and the matching text from the dataframe to the results dataframe
matches_df['Line'] = [corrected_lines[i] for i in matches_df['LineIndex']]
matches_df['BestMatchText'] = [df.iloc[i]['concatenated_fields'] for i in matches_df['BestMatchRowIndex']]


df['screening2'] = df.index.isin(matches_df['BestMatchRowIndex'])
df.screening2.value_counts()


df.loc[df['screening2'] == True, 'screening1'] = True
print(df['screening1'].value_counts())
print(df['screening2'].value_counts())
print(df.groupby('screening1')['screening2'].value_counts())
print(df.groupby('screening1')['screening2'].value_counts().sum())

# Counting NaNs per column
nan_count = df.isna().sum()

print("\nNumber of NaNs per column:\n")
print(nan_count)
# df.to_pickle(f"{content_dir}preprocessed_articles.pkl")

NameError: name 'df' is not defined

In [5]:
df['uniqueid'] = df.index.astype(str)
# find indeces with no title or abstract or is nan
no_title_abstract = df[(df['titles'] == '') | (df['abstract'] == '') | (df['titles'].isna()) | (df['abstract'].isna())]
no_title_abstract.to_pickle(f"{content_dir}no_title_abstract.pkl")
df.drop("uniqueid", axis=1, inplace=True)

print(len(df),len(no_title_abstract))
full_records = df[~df.index.isin(no_title_abstract.index)]

print(full_records['screening1'].value_counts())
print(full_records.groupby('screening1')['screening2'].value_counts())
full_records.to_pickle("preprocessed_articles_filtered.pkl")
full_records.to_excel("preprocessed_articles_filtered.xlsx")


4662 161
screening1
False    4399
True      102
Name: count, dtype: int64
screening1  screening2
False       False         4399
True        True           102
Name: count, dtype: int64


In [23]:
full_records

,record,database,source-app,rec-number,foreign-keys,key,ref-type,contributors,titles,title,...,related-urls,url,custom6,orig-pub,publisher,secondary-authors,translated-title,concatenated_fields,screening1,screening2
0,All_incl_articles_20230630.enlEndNote1117Clini...,All_incl_articles_20230630.enl,EndNote,1,1,1,17,,Clinical practice guidelines for the diagnosis...,Clinical practice guidelines for the diagnosis...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Clinical practice guidelines for the diagnosi...,False,False
1,All_incl_articles_20230630.enlEndNote2217Osteo...,All_incl_articles_20230630.enl,EndNote,2,2,2,17,,"Osteoporosis prevention, diagnosis, and therap...","Osteoporosis prevention, diagnosis, and therapy",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Osteoporosis prevention, diagnosis, and thera...",False,False
2,All_incl_articles_20230630.enlEndNote3317Obesi...,All_incl_articles_20230630.enl,EndNote,3,3,3,17,,Obesity: preventing and managing the global ep...,Obesity: preventing and managing the global ep...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Obesity: preventing and managing the global e...,False,False
3,All_incl_articles_20230630.enlEndNote4417Unite...,All_incl_articles_20230630.enl,EndNote,4,4,4,17,,United Kingdom back pain exercise and manipula...,United Kingdom back pain exercise and manipula...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,United Kingdom back pain exercise and manipul...,False,False
4,All_incl_articles_20230630.enlEndNote5517Physi...,All_incl_articles_20230630.enl,EndNote,5,5,5,17,,Physiotherapy rehabilitation after total knee ...,Physiotherapy rehabilitation after total knee ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Physiotherapy rehabilitation after total knee...,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3152,All_incl_articles_20230630.enlEndNote272727271...,All_incl_articles_20230630.enl,EndNote,2727,2727,2727,17,"Østerås, N.Hagen, K. B.Grotle, M.Sand-Svartrud...",Exercise programme with telephone follow-up fo...,Exercise programme with telephone follow-up fo...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Østerås, N.Hagen, K. B.Grotle, M.Sand-Svartrud...",False,False
3153,All_incl_articles_20230630.enlEndNote272827281...,All_incl_articles_20230630.enl,EndNote,2728,2728,2728,17,"Østerås, N.van Bodegom-Vos, L.Dziedzic, K.Mose...",Implementing international osteoarthritis trea...,Implementing international osteoarthritis trea...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Østerås, N.van Bodegom-Vos, L.Dziedzic, K.Mose...",False,False
3165,All_incl_articles_20230630.enlEndNote454945491...,All_incl_articles_20230630.enl,EndNote,4549,4549,4549,17,"Õunpuu, SylviaPierz, KristanRethlefsen, Susan ...",Cost savings for single event multilevel surge...,Cost savings for single event multilevel surge...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Õunpuu, SylviaPierz, KristanRethlefsen, Susan ...",False,False
3889,All_incl_articles_20230630.enlEndNote338533851...,All_incl_articles_20230630.enl,EndNote,3385,3385,3385,17,"Śliwiński, Z.Frączek, E.Starczyńska, M.",Role of the physiotherapist in the orthopaedic...,Role of the physiotherapist in the orthopaedic...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Śliwiński, Z.Frączek, E.Starczyńska, M. Role o...",False,False


## Checking PDFs

In [5]:
import pandas as pd
import sys
import numpy as np
sys.path.append("/home/bbb1417/academatepy/app")
from academate import academate


workdir = "/home/bbb1417/academatepy/validation/PICOS/"
df = pd.read_pickle(f'{workdir}preprocessed_articles_filtered.pkl')
df['predicted_screening1'] = df['screening1']
df['predicted_screening1'].value_counts()
# df['pdf_path'] = "/home/bbb1417/academatepy/validation/PICOS/pdfs/" + df['pdf_name']
df['pdf_path'] = f"{workdir}/pdfs/" + df['pdf_name']

df.columns

Index(['record', 'database', 'source-app', 'rec-number', 'foreign-keys', 'key',
       'ref-type', 'contributors', 'titles', 'title', 'style',
       'secondary-title', 'short-title', 'pages', 'volume', 'number',
       'edition', 'keywords', 'keyword', 'dates', 'year', 'pub-dates', 'date',
       'isbn', 'accession-num', 'abstract', 'notes', 'urls', 'custom2',
       'remote-database-provider', 'language', 'electronic-resource-num',
       'auth-address', 'pub-location', 'remote-database-name', 'authors',
       'author', 'related-urls', 'url', 'custom6', 'orig-pub', 'publisher',
       'secondary-authors', 'translated-title', 'concatenated_fields',
       'screening1', 'screening2', 'pdf_name', 'predicted_screening1',
       'pdf_path'],
      dtype='object')

In [6]:
df['record_clean'] = df['record'].str.extract(r'(All_incl_articles_20230630\.enlEndNote\d+)')


pdfs_downloaded = pd.read_excel(f"{workdir}/final_papers_data_cleaned_handEdition.ods", engine="odf", sheet_name="Sheet1")
pdfs_downloaded['pdf_name'] = pdfs_downloaded['filepath'].str.split("\\").str[-1]
pdfs_downloaded['pdf_name'] = pdfs_downloaded['pdf_name'].fillna(pdfs_downloaded['Filename (without suffix)'].astype(str) + ".pdf")
pdfs_downloaded['pdf_name'] = pdfs_downloaded['pdf_name'].replace('nan.pdf', np.nan)
pdfs_downloaded['record_clean'] = pdfs_downloaded['record'].str.extract(r'(All_incl_articles_20230630\.enlEndNote\d+)')
rec2pdf_name = dict(zip(pdfs_downloaded.contributors, pdfs_downloaded.pdf_name))
pdfs_downloaded['pdf_name'].isna().value_counts()

df['pdf_name']=df.contributors.map(rec2pdf_name)
df['pdf_path']=workdir + 'pdfs/' + df['pdf_name']

records = pd.read_pickle("/home/bbb1417/academatepy/validation/PICOS/results/results_finalend_athene-70b_latest/results_screening1.pkl")
rec2uniqueid = dict(zip(records.contributors, records.uniqueid))

df['uniqueid']=df.contributors.map(rec2uniqueid)

print(df[df.screening1==True].sort_values('pdf_name')['pdf_name'].isna().value_counts())
print(df['uniqueid'].isna().value_counts())

df[df.screening1==True].sort_values('pdf_name')

pdf_name
True     135
False     37
Name: count, dtype: int64
uniqueid
False    4501
Name: count, dtype: int64


,record,database,source-app,rec-number,foreign-keys,key,ref-type,contributors,titles,title,...,secondary-authors,translated-title,concatenated_fields,screening1,screening2,pdf_name,predicted_screening1,pdf_path,record_clean,uniqueid
329,All_incl_articles_20230630.enlEndNote28128117B...,All_incl_articles_20230630.enl,EndNote,281,281,281,17,"Bendix, T.Bendix, A.Labriola, M.Haestrup, C.Eb...",Functional restoration versus outpatient physi...,Functional restoration versus outpatient physi...,...,NaN,NaN,"Bendix, T.Bendix, A.Labriola, M.Haestrup, C.Eb...",True,False,00007632-200010010-00012.pdf,True,/home/bbb1417/academatepy/validation/PICOS/pdf...,All_incl_articles_20230630.enlEndNote28128117,61594e276af61b93bac7
231,All_incl_articles_20230630.enlEndNote407440741...,All_incl_articles_20230630.enl,EndNote,4074,4074,4074,17,"Bakker, C.Hidding, A.van der Linden, S.van Doo...",Cost effectiveness of group physical therapy c...,Cost effectiveness of group physical therapy c...,...,NaN,NaN,"Bakker, C.Hidding, A.van der Linden, S.van Doo...",True,False,10.1002_art.1790060303.pdf,True,/home/bbb1417/academatepy/validation/PICOS/pdf...,All_incl_articles_20230630.enlEndNote4074407417,92d727d24a897470caad
1716,All_incl_articles_20230630.enlEndNote147214721...,All_incl_articles_20230630.enl,EndNote,1472,1472,1472,17,"Hlobil, H.Uegaki, K.Staal, J. B.de Bruyne, M. ...",Substantial sick-leave costs savings due to a ...,Substantial sick-leave costs savings due to a ...,...,NaN,NaN,"Hlobil, H.Uegaki, K.Staal, J. B.de Bruyne, M. ...",True,True,10.1007_s00586-006-0283-9.pdf,True,/home/bbb1417/academatepy/validation/PICOS/pdf...,All_incl_articles_20230630.enlEndNote1472147217,3bda9ac79e3914a967bd
551,All_incl_articles_20230630.enlEndNote47647617B...,All_incl_articles_20230630.enl,EndNote,476,476,476,17,"Burton, A. K.Balagué, F.Cardon, G.Eriksen, H. ...",How to prevent low back painBest Pract Res Cli...,How to prevent low back pain,...,NaN,NaN,"Burton, A. K.Balagué, F.Cardon, G.Eriksen, H. ...",True,True,10.1007_s00586-006-1070-3.pdf,True,/home/bbb1417/academatepy/validation/PICOS/pdf...,All_incl_articles_20230630.enlEndNote47647617,8196c756024e004577cf
493,All_incl_articles_20230630.enlEndNote42742717B...,All_incl_articles_20230630.enl,EndNote,427,427,427,17,"Brown, K. C.Sirles, A. T.Hilyer, J. C.Thomas, ...",Cost-effectiveness of a back school interventi...,Cost-effectiveness of a back school interventi...,...,NaN,NaN,"Brown, K. C.Sirles, A. T.Hilyer, J. C.Thomas, ...",True,False,10.1097_00007632-199210000-00016.pdf,True,/home/bbb1417/academatepy/validation/PICOS/pdf...,All_incl_articles_20230630.enlEndNote42742717,479f96fd65da99f43936
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4488,All_incl_articles_20230630.enlEndNote390739071...,All_incl_articles_20230630.enl,EndNote,3907,3907,3907,17,"Williams, A.van Dongen, J. M.Kamper, S. J.O'Br...",Economic evaluation of a healthy lifestyle int...,Economic evaluation of a healthy lifestyle int...,...,NaN,NaN,"Williams, A.van Dongen, J. M.Kamper, S. J.O'Br...",True,False,NaN,True,NaN,All_incl_articles_20230630.enlEndNote3907390717,c7903ef89e27ab97ac36
4540,All_incl_articles_20230630.enlEndNote395039501...,All_incl_articles_20230630.enl,EndNote,3950,3950,3950,17,"Woods, B.Manca, A.Weatherly, H.Saramago, P.Sid...",Cost-effectiveness of adjunct non-pharmacologi...,Cost-effectiveness of adjunct non-pharmacologi...,...,NaN,NaN,"Woods, B.Manca, A.Weatherly, H.Saramago, P.Sid...",True,False,NaN,True,NaN,All_incl_articles_20230630.enlEndNote3950395017,a7eb5b38e7cea30ea0ad
4583,All_incl_articles_20230630.enlEndNote398439841...,All_incl_articles_20230630.enl,EndNote,3984,3984,3984,17,"Yates, C. J.Chauchard, M. A.Liew, D.Bucknill, ...",Bridging the osteoporosis treatment gap: perfo...,Bridging the osteoporosis treatment gap: perfo...,...,NaN,NaN,"Yates, C. J.Chauchard, M. A.Liew, D.Bucknill, ...",True,False,NaN,True,NaN,All_incl_articles_20230630.enlEndNote3984398417,dcd190eaa61

In [10]:
df['record_clean'] = df['record'].str.extract(r'(All_incl_articles_20230630\.enlEndNote\d+)')

pdfs_downloaded = pd.read_excel(f"{workdir}/final_deduplicated_dataframe.xlsx", sheet_name="Sheet1")
pdfs_downloaded['filename'] = pdfs_downloaded['filename'].apply(lambda x: x + ".pdf" if not x.endswith(".pdf") else x)
pdfs_downloaded['pdf_name'] = pdfs_downloaded['filename'].astype(str)
pdfs_downloaded['pdf_path'] = workdir + 'pdfs/' + pdfs_downloaded['pdf_name']

# failing_pdfs ={'bf28718d03b917546687',
#                '2592577458d22664dea4'
#               }

failing_pdfs={}

pdfs_downloaded=pdfs_downloaded[~pdfs_downloaded.uniqueid.isin(failing_pdfs)]

In [11]:
id2pdf = dict(zip(pdfs_downloaded.uniqueid, pdfs_downloaded.pdf_name))
df['pdf_name'] = df['uniqueid'].map(id2pdf)
df['pdf_path'] = f"{workdir}/pdfs/" + df['pdf_name']

In [12]:
print(df[df.screening1==True].sort_values('pdf_name')['pdf_name'].isna().value_counts())
print(df['uniqueid'].isna().value_counts())

pdf_name
False    171
True       1
Name: count, dtype: int64
uniqueid
False    4501
Name: count, dtype: int64


In [17]:
print(df.columns)
df[df['screening1']==True].sort_values(['screening1', 'screening2'], ascending=False)

Index(['record', 'database', 'source-app', 'rec-number', 'foreign-keys', 'key',
       'ref-type', 'contributors', 'titles', 'title', 'style',
       'secondary-title', 'short-title', 'pages', 'volume', 'number',
       'edition', 'keywords', 'keyword', 'dates', 'year', 'pub-dates', 'date',
       'isbn', 'accession-num', 'abstract', 'notes', 'urls', 'custom2',
       'remote-database-provider', 'language', 'electronic-resource-num',
       'auth-address', 'pub-location', 'remote-database-name', 'authors',
       'author', 'related-urls', 'url', 'custom6', 'orig-pub', 'publisher',
       'secondary-authors', 'translated-title', 'concatenated_fields',
       'screening1', 'screening2', 'pdf_name', 'predicted_screening1',
       'pdf_path', 'record_clean', 'uniqueid'],
      dtype='object')


,record,database,source-app,rec-number,foreign-keys,key,ref-type,contributors,titles,title,...,secondary-authors,translated-title,concatenated_fields,screening1,screening2,pdf_name,predicted_screening1,pdf_path,record_clean,uniqueid
19,All_incl_articles_20230630.enlEndNote191917Abb...,All_incl_articles_20230630.enl,EndNote,19,19,19,17,"Abbott, J. H.Wilson, R.Pinto, D.Chapple, C. M....",Incremental clinical effectiveness and cost ef...,Incremental clinical effectiveness and cost ef...,...,NaN,NaN,"Abbott, J. H.Wilson, R.Pinto, D.Chapple, C. M....",True,True,j.joca.2018.12.004.pdf,True,/home/bbb1417/academatepy/validation/PICOS//pd...,All_incl_articles_20230630.enlEndNote191917,50fb66a61fff1cf5eaa8
24,All_incl_articles_20230630.enlEndNote212117Abo...,All_incl_articles_20230630.enl,EndNote,21,21,21,17,"Aboagye, E.Karlsson, M. L.Hagberg, J.Jensen, I.",Cost-effectiveness of early interventions for ...,Cost-effectiveness of early interventions for ...,...,NaN,NaN,"Aboagye, E.Karlsson, M. L.Hagberg, J.Jensen, I...",True,True,10.2340_16501977-1910.pdf,True,/home/bbb1417/academatepy/validation/PICOS//pd...,All_incl_articles_20230630.enlEndNote212117,40c095e2133bcd8d675f
148,All_incl_articles_20230630.enlEndNote12412417A...,All_incl_articles_20230630.enl,EndNote,124,124,124,17,"Ankjaer-Jensen, A.Manniche, C.Nielsen, H.",[Postoperative rehabilitation of patients oper...,[Postoperative rehabilitation of patients oper...,...,NaN,NaN,"Ankjaer-Jensen, A.Manniche, C.Nielsen, H. [Pos...",True,True,10.1503_cjs.006523.pdf,True,/home/bbb1417/academatepy/validation/PICOS//pd...,All_incl_articles_20230630.enlEndNote12412417,91bff4b4c89e9ca278a7
157,All_incl_articles_20230630.enlEndNote13213217A...,All_incl_articles_20230630.enl,EndNote,132,132,132,17,"Apeldoorn, A. T.Bosmans, J. E.Ostelo, R. W.de ...",Cost-effectiveness of a classification-based s...,Cost-effectiveness of a classification-based s...,...,NaN,NaN,"Apeldoorn, A. T.Bosmans, J. E.Ostelo, R. W.de ...",True,True,586_2011_Article_2144.pdf,True,/home/bbb1417/academatepy/validation/PICOS//pd...,All_incl_articles_20230630.enlEndNote13213217,d60f6194b279c6deb1d6
249,All_incl_articles_20230630.enlEndNote21221217B...,All_incl_articles_20230630.enl,EndNote,212,212,212,17,"Barker, K. L.Newman, M.Stallard, N.Leal, J.Low...",Physiotherapy rehabilitation for osteoporotic ...,Physiotherapy rehabilitation for osteoporotic ...,...,NaN,NaN,"Barker, K. L.Newman, M.Stallard, N.Leal, J.Low...",True,True,Physiotherapy_Rehabilitation_for_Osteoporotic_...,True,/home/bbb1417/academatepy/validation/PICOS//pd...,All_incl_articles_20230630.enlEndNote21221217,aa66d64dc44217315fd5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4488,All_incl_articles_20230630.enlEndNote390739071...,All_incl_articles_20230630.enl,EndNote,3907,3907,3907,17,"Williams, A.van Dongen, J. M.Kamper, S. J.O'Br...",Economic evaluation of a healthy lifestyle int...,Economic evaluation of a healthy lifestyle int...,...,NaN,NaN,"Williams, A.van Dongen, J. M.Kamper, S. J.O'Br...",True,False,Williams_2019_Economic_evaluation_of_a_healthy...,True,/home/bbb1417/academatepy/validation/PICOS//pd...,All_incl_articles_20230630.enlEndNote3907390717,c7903ef89e27ab97ac36
4540,All_incl_articles_20230630.enlEndNote395039501...,All_incl_articles_20230630.enl,EndNote,3950,3950,3950,17,"Woods, B.Manca, A.Weatherly, H.Saramago, P.Sid...",Cost-effectiveness of adjunct non-pharmacologi...,Cost-effectiveness of adjunct non-pharmacologi...,...,NaN,NaN,"Woods, B.Manca, A.Weatherly, H.Saramago, P.Sid...",True,False,10.1371_journal.pone.0172749.pdf,True,/home/bbb1417/academatepy/validation/PICOS//pd...,All_incl_articles_20230630.enlEndNote3950395017,a7eb5b38e7cea30ea0ad
4583,All_incl_articles_20230630.enlEndNote398439841...,All_incl_articles_20230630.enl,EndNote,3984,3984,3984,17,"Yates, C. J.Chauchard, M. A.Liew, D.Bucknill, ...",Bridging the osteoporosis treatment gap: perfo...,Bridging the osteoporosis treatment gap: perfo...,

In [21]:
df.drop('uniqueid', axis=1, inplace=True)
print(df.columns)

df.to_pickle(f"{workdir}withpdfs_preprocessed_articles_filtered.pkl")

Index(['record', 'database', 'source-app', 'rec-number', 'foreign-keys', 'key',
       'ref-type', 'contributors', 'titles', 'title', 'style',
       'secondary-title', 'short-title', 'pages', 'volume', 'number',
       'edition', 'keywords', 'keyword', 'dates', 'year', 'pub-dates', 'date',
       'isbn', 'accession-num', 'abstract', 'notes', 'urls', 'custom2',
       'remote-database-provider', 'language', 'electronic-resource-num',
       'auth-address', 'pub-location', 'remote-database-name', 'authors',
       'author', 'related-urls', 'url', 'custom6', 'orig-pub', 'publisher',
       'secondary-authors', 'translated-title', 'concatenated_fields',
       'screening1', 'screening2', 'pdf_name', 'predicted_screening1',
       'pdf_path', 'record_clean'],
      dtype='object')


In [83]:
import pickle

with open(f"{workdir}id2pdf.pkl", 'wb') as file:  # Open in binary write mode ('wb')
      pickle.dump(id2pdf, file)

In [84]:
screening1_mistaken = {"2a2e02de1d82911e32a6",
                       "6f7a45fb2bcdadd39842",
                       "2e9c54d9ad3b0950ab1e"}
                       

# Ensure 'screening1' is a boolean column (if not already)
if 'screening1' not in df.columns or df['screening1'].dtype != 'bool':
    df['screening1'] = False  # Initialize with default False

# Update 'screening1' to True for the failing PDFs
df.loc[df['uniqueid'].isin(screening1_mistaken), 'screening1'] = True

In [85]:
df.dropna(subset=['pdf_name'], inplace=True)
df['pdf_path'] = f"{workdir}/pdfs/" + df['pdf_name']
df.screening1.value_counts()

screening1
True    172
Name: count, dtype: int64

In [90]:
df[df.uniqueid.duplicated()]

,record,database,source-app,rec-number,foreign-keys,key,ref-type,contributors,titles,title,...,secondary-authors,translated-title,concatenated_fields,screening1,screening2,pdf_name,predicted_screening1,pdf_path,record_clean,uniqueid
166,All_incl_articles_20230630.enlEndNote14014017A...,All_incl_articles_20230630.enl,EndNote,140,140,140,17,"Aras, E. L.Bunger, C.Hansen, E. S.Søgaard, R.",Health-care costs of conservative management o...,Health-care costs of conservative management o...,...,NaN,NaN,"Aras, E. L.Bunger, C.Hansen, E. S.Søgaard, R. ...",True,False,brs.0000000000001219.pdf,False,/home/bbb1417/academatepy/validation/PICOS//pd...,All_incl_articles_20230630.enlEndNote14014017,2a2e02de1d82911e32a6
3445,All_incl_articles_20230630.enlEndNote298929891...,All_incl_articles_20230630.enl,EndNote,2989,2989,2989,17,"Rantonen, J.Karppinen, J.Vehtari, A.Luoto, S.V...",Effectiveness of three interventions for secon...,Effectiveness of three interventions for secon...,...,NaN,NaN,"Rantonen, J.Karppinen, J.Vehtari, A.Luoto, S.V...",True,False,10.1186_s12889-016-2974-4.pdf,False,/home/bbb1417/academatepy/validation/PICOS//pd...,All_incl_articles_20230630.enlEndNote2989298917,6f7a45fb2bcdadd39842
4168,All_incl_articles_20230630.enlEndNote362736271...,All_incl_articles_20230630.enl,EndNote,3627,3627,3627,17,"Timm, K. E.",The clinical and cost-effectiveness of two dif...,The clinical and cost-effectiveness of two dif...,...,NaN,NaN,"Timm, K. E. The clinical and cost-effectivenes...",True,False,Timm_1997_The_clinical_and_cost_effectiveness.pdf,True,/home/bbb1417/academatepy/validation/PICOS//pd...,All_incl_articles_20230630.enlEndNote3627362717,2e9c54d9ad3b0950ab1e


# Once more 

In [2]:
sys.path.append("/home/bbb1417/academatepy/app")
from academate import academate
import pandas as pd
import sys
import numpy as np

workdir = "/home/bbb1417/academatepy/validation/PICOS/"
df = pd.read_pickle(f'{workdir}july_preprocessed_articles_filtered.pkl')
df.columns

Index(['record', 'database', 'source-app', 'rec-number', 'foreign-keys', 'key',
       'ref-type', 'contributors', 'titles', 'title', 'style',
       'secondary-title', 'short-title', 'pages', 'volume', 'number',
       'edition', 'keywords', 'keyword', 'dates', 'year', 'pub-dates', 'date',
       'isbn', 'accession-num', 'abstract', 'notes', 'urls', 'custom2',
       'remote-database-provider', 'language', 'electronic-resource-num',
       'auth-address', 'pub-location', 'remote-database-name', 'authors',
       'author', 'related-urls', 'url', 'custom6', 'orig-pub', 'publisher',
       'secondary-authors', 'translated-title', 'concatenated_fields',
       'screening1', 'screening2'],
      dtype='object')

In [4]:
df.sort_values(['screening1', 'screening2'], ascending=False)

,record,database,source-app,rec-number,foreign-keys,key,ref-type,contributors,titles,title,...,related-urls,url,custom6,orig-pub,publisher,secondary-authors,translated-title,concatenated_fields,screening1,screening2
4272,All_incl_articles_20230630.enlEndNote421542151...,All_incl_articles_20230630.enl,EndNote,4215,4215,4215,17,"van den Hout, Wilbert B.Peul, Wilco C.Koes, Ba...",Prolonged conservative care versus early surge...,Prolonged conservative care versus early surge...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"van den Hout, Wilbert B.Peul, Wilco C.Koes, Ba...",True,True
4267,All_incl_articles_20230630.enlEndNote371637161...,All_incl_articles_20230630.enl,EndNote,3716,3716,3716,17,"van de Graaf, V. A.van Dongen, J. M.Willigenbu...",How do the costs of physical therapy and arthr...,How do the costs of physical therapy and arthr...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"van de Graaf, V. A.van Dongen, J. M.Willigenbu...",True,True
4452,All_incl_articles_20230630.enlEndNote387538751...,All_incl_articles_20230630.enl,EndNote,3875,3875,3875,17,"Westmoreland, J. L.Williams, N. H.Wilkinson, C...",Should your GP be an osteopath? Patients' view...,Should your GP be an osteopath? Patients' view...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Westmoreland, J. L.Williams, N. H.Wilkinson, C...",True,True
4261,All_incl_articles_20230630.enlEndNote371137111...,All_incl_articles_20230630.enl,EndNote,3711,3711,3711,17,"Van Belleghem, G.Van Deynse, H.Devos, S.Huysma...",Health care utilization after hospitalization ...,Health care utilization after hospitalization ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Van Belleghem, G.Van Deynse, H.Devos, S.Huysma...",True,True
4257,All_incl_articles_20230630.enlEndNote370737071...,All_incl_articles_20230630.enl,EndNote,3707,3707,3707,17,"Valentini, A.Federici, M.Cianfarani, M. A.Tara...",Frailty and nutritional status in older people...,Frailty and nutritional status in older people...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Valentini, A.Federici, M.Cianfarani, M. A.Tara...",True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6,All_incl_articles_20230630.enlEndNote7717Intra...,All_incl_articles_20230630.enl,EndNote,7,7,7,17,,Intra-articular viscosupplementation with hyla...,Intra-articular viscosupplementation with hyla...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Intra-articular viscosupplementation with hyl...,False,False
11,All_incl_articles_20230630.enlEndNote121217Exe...,All_incl_articles_20230630.enl,EndNote,12,12,12,17,,Exercise for Knee Injury Prevention: A Summary...,Exercise for Knee Injury Prevention: A Summary...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Exercise for Knee Injury Prevention: A Summar...,False,False
0,All_incl_articles_20230630.enlEndNote1117Clini...,All_incl_articles_20230630.enl,EndNote,1,1,1,17,,Clinical practice guidelines for the diagnosis...,Clinical practice guidelines for the diagnosis...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Clinical practice guidelines for the diagnosi...,False,False
5,All_incl_articles_20230630.enlEndNote6617Arthr...,All_incl_articles_20230630.enl,EndNote,6,6,6,17,,Arthroscopic lavage and debridement for osteoa...,Arthroscopic lavage and debridement for osteoa...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Arthroscopic lavage and debridement for osteo...,False,False
